# Import necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import math
from math import radians, sin, cos, sqrt, atan2
import warnings
warnings.filterwarnings('ignore')

---
# Current baseline parameters

In [ ]:
lat_ref = -29.79165152  # Current DC latitude
lon_ref = 151.1250996   # Current DC longitude
avg_inv_cbm_curr = 14898  # Current inventory CBM
storage_cost_cbm_curr = 30  # Current storage cost per CBM
handling_cost_cbm_curr = 10  # Current handling cost per CBM
TRUCK_CAPACITY = 20000  # 20 tons truck capacity (Standard Australian heavy truck)

---
Connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


---
## df_shipto_master

In [ ]:
print("\n📊Preparing data for clustering...")

file_path_shipto_master = "/content/drive/MyDrive/0027. CUỘC THI/01. LOGAGE 2025/02. VÒNG 2/PROJECT/02. IMPLEMENTATION/03. New distribution network design/processed_shipto_master.csv"

df_shipto_master = pd.read_csv(file_path_shipto_master)


📊Preparing data for clustering...


In [ ]:
display(df_shipto_master.head(3))

,ShipToID,Longitude,Latitude,State,City,Suburb,Annual_CBM_Demand_ShipTo,Annual_KG_Demand_ShipTo,Total_Shipments
0,3001BR,151.001259,-33.797556,New South Wales,Parramatta,North Parramatta,91.182518,11801.9360,313
1,3002BR,149.206300,-35.347000,New South Wales,Queanbeyan,Queanbeyan,100.414742,12320.3690,314
2,3003BR,147.343011,-35.122016,New South Wales,Wagga Wagga,Wagga Wagga,882.881842,99185.9602,694


---
# Divided into 5 phases for delivery

In [ ]:
# Chia 5 xuống
df_shipto_master['Annual_KG_Demand_ShipTo'] = df_shipto_master['Annual_KG_Demand_ShipTo'] / 5
df_shipto_master['Annual_CBM_Demand_ShipTo'] = df_shipto_master['Annual_CBM_Demand_ShipTo'] / 5
df_shipto_master['Total_Shipments'] = df_shipto_master['Total_Shipments'] / 5

In [ ]:
# Enhanced KG to CBM conversion ratio
if 'Annual_CBM_Demand_ShipTo' in df_shipto_master.columns:
    total_kg = df_shipto_master['Annual_KG_Demand_ShipTo'].sum()
    total_cbm = df_shipto_master['Annual_CBM_Demand_ShipTo'].sum()
    ACTUAL_KG_PER_CBM = total_kg / total_cbm if total_cbm > 0 else 167
    print(f"✅ Using actual KG/CBM ratio from data: {ACTUAL_KG_PER_CBM:.1f} kg/CBM")
else:
    ACTUAL_KG_PER_CBM = 167  # Realistic for Australian freight
    print(f"📊 Using industry standard KG/CBM ratio: {ACTUAL_KG_PER_CBM} kg/CBM")

# Enhanced cost adjustment factors
URBAN_ADJUSTMENT_FACTOR = 1.1  # Urban areas 10% higher costs
RURAL_ADJUSTMENT_FACTOR = 0.7  # Rural areas 30% lower costs

✅ Using actual KG/CBM ratio from data: 143.6 kg/CBM


---
## cost curve data

In [ ]:
file_path_cost_curve = '/content/drive/MyDrive/0027. CUỘC THI/01. LOGAGE 2025/02. VÒNG 2/PROJECT/01. DATASET/LOGage2025_Round 2_Data.xlsx_CostCurve.csv'

costcurve = pd.read_csv(file_path_cost_curve)

display(costcurve)

,State,Mode,Intercept (AUD),Slope (AUD/KM),Min Distance (KM),Max Distance (KM)
0,New South Wales,FTL,"225,2","0,9",0,55
1,New South Wales,LTL,"360,4","1,1",55,999999
2,Queensland,FTL,"192,6","0,2",0,100
3,Queensland,LTL,"226,7","0,3",100,999999
4,Victoria,FTL,"198,4",1,0,70
5,Victoria,LTL,"334,8","0,2",70,999999
6,Western Australia,FTL,"95,8","1,2",0,55
7,Western Australia,LTL,"242,2","0,5",55,999999


In [ ]:
print(f"🏢 Current DC: ({lon_ref}, {lat_ref})")
print(f"🚛 Truck capacity: {TRUCK_CAPACITY:,} KG (Standard heavy truck capacity in Australia)")
print(f"📦 Current inventory: {avg_inv_cbm_curr:,} CBM")
print(f"⚖️ KG to CBM ratio: {ACTUAL_KG_PER_CBM} kg/CBM (Industry standard assumption)")
baseline_total_cost =  3078427
print(f"💰 Baseline total cost: ${baseline_total_cost:,.0f}")

🏢 Current DC: (151.1250996, -29.79165152)
🚛 Truck capacity: 20,000 KG (Standard heavy truck capacity in Australia)
📦 Current inventory: 14,898 CBM
⚖️ KG to CBM ratio: 143.64973855978565 kg/CBM (Industry standard assumption)
💰 Baseline total cost: $3,078,427


---
# Function

In [ ]:
def lookup_cost_enhanced(state, distance, costcurve):
    """Enhanced cost curve lookup with if-else logic for better accuracy"""
    try:
        # Clean and standardize state name
        state_clean = state.strip().upper()

        # Handle different state variations and apply cost curves
        if state_clean in ['NEW SOUTH WALES', 'NSW']:
            if distance <= 55:
                return 225.2, 0.9, 'FTL'
            else:
                return 360.4, 1.1, 'LTL'

        elif state_clean in ['QUEENSLAND', 'QLD']:
            if distance <= 100:
                return 192.6, 0.2, 'FTL'
            else:
                return 226.7, 0.3, 'LTL'

        elif state_clean in ['VICTORIA', 'VIC']:
            if distance <= 70:
                return 198.4, 1.0, 'FTL'
            else:
                return 334.8, 0.2, 'LTL'

        elif state_clean in ['WESTERN AUSTRALIA', 'WA']:
            if distance <= 55:
                return 95.8, 1.2, 'FTL'
            else:
                return 242.2, 0.5, 'LTL'

        # Handle other Australian states with fallback to NSW
        elif state_clean in ['SOUTH AUSTRALIA', 'SA', 'TASMANIA', 'TAS', 'NORTHERN TERRITORY', 'NT', 'AUSTRALIAN CAPITAL TERRITORY', 'ACT']:
            print(f"⚠️ Using NSW fallback for {state}")
            if distance <= 55:
                return 225.2, 0.9, 'FTL'
            else:
                return 360.4, 1.1, 'LTL'

        # Try partial matching for state names
        else:
            # Extract first word for partial matching
            first_word = state.split()[0].upper() if state.split() else state_clean

            if 'NEW' in first_word or 'SOUTH' in state_clean.split():
                if distance <= 55:
                    return 225.2, 0.9, 'FTL'
                else:
                    return 360.4, 1.1, 'LTL'

            elif 'QUEEN' in first_word:
                if distance <= 100:
                    return 192.6, 0.2, 'FTL'
                else:
                    return 226.7, 0.3, 'LTL'

            elif 'VICT' in first_word:
                if distance <= 70:
                    return 198.4, 1.0, 'FTL'
                else:
                    return 334.8, 0.2, 'LTL'

            elif 'WEST' in first_word:
                if distance <= 55:
                    return 95.8, 1.2, 'FTL'
                else:
                    return 242.2, 0.5, 'LTL'

            # Ultimate fallback to NSW LTL
            else:
                print(f"⚠️ Using NSW LTL fallback for unrecognized state: {state}")
                return 360.4, 1.1, 'LTL'

    except Exception as e:
        print(f"⚠️ Critical error in cost lookup for {state}: {e}")
        # Ultimate fallback to NSW LTL
        return 360.4, 1.1, 'LTL'

def enhanced_truck_consolidation(rdc_shiptos, rdc_id, TRUCK_CAPACITY):
    """Enhanced truck consolidation algorithm with better bin packing"""
    if len(rdc_shiptos) == 0:
        return []

    # Sort by demand in descending order for better bin packing (First Fit Decreasing)
    remaining_shiptos = rdc_shiptos.copy().sort_values('Annual_KG_Demand_ShipTo', ascending=False).reset_index(drop=True)
    trucks = []
    truck_counter = 0

    while len(remaining_shiptos) > 0:
        # Create new truck
        truck_shiptos = []
        current_weight = 0
        indices_to_remove = []

        # Try to fit shipments into current truck using First Fit Decreasing
        for idx, row in remaining_shiptos.iterrows():
            shipment_weight = row['Annual_KG_Demand_ShipTo']

            # Skip if shipment weight is 0 or negative
            if shipment_weight <= 0:
                indices_to_remove.append(idx)
                continue

            # If shipment fits completely in truck
            if current_weight + shipment_weight <= TRUCK_CAPACITY:
                truck_shiptos.append({
                    'ShipToID': row['ShipToID'],
                    'Longitude': row['Longitude'],
                    'Latitude': row['Latitude'],
                    'Annual_KG_Demand_ShipTo': shipment_weight
                })
                current_weight += shipment_weight
                indices_to_remove.append(idx)

            # If shipment is larger than remaining capacity but truck has space
            elif shipment_weight > TRUCK_CAPACITY and current_weight < TRUCK_CAPACITY:
                # Split the shipment: take what fits in current truck
                remaining_capacity = TRUCK_CAPACITY - current_weight
                if remaining_capacity > 0:
                    truck_shiptos.append({
                        'ShipToID': f"{row['ShipToID']}_split_{truck_counter}",
                        'Longitude': row['Longitude'],
                        'Latitude': row['Latitude'],
                        'Annual_KG_Demand_ShipTo': remaining_capacity
                    })
                    current_weight += remaining_capacity

                    # Reduce the original shipment weight
                    remaining_shiptos.loc[idx, 'Annual_KG_Demand_ShipTo'] -= remaining_capacity
                break  # Truck is now full

            # If single shipment exceeds truck capacity and truck is empty
            elif shipment_weight > TRUCK_CAPACITY and current_weight == 0:
                # Create dedicated truck for this oversized shipment
                truck_shiptos.append({
                    'ShipToID': f"{row['ShipToID']}_oversized_{truck_counter}",
                    'Longitude': row['Longitude'],
                    'Latitude': row['Latitude'],
                    'Annual_KG_Demand_ShipTo': min(shipment_weight, TRUCK_CAPACITY)
                })
                current_weight = min(shipment_weight, TRUCK_CAPACITY)

                # If shipment is larger than truck capacity, reduce it
                if shipment_weight > TRUCK_CAPACITY:
                    remaining_shiptos.loc[idx, 'Annual_KG_Demand_ShipTo'] -= TRUCK_CAPACITY
                else:
                    indices_to_remove.append(idx)
                break  # Move to next truck

        # Only create truck if it has shipments
        if truck_shiptos:
            trucks.append({
                'truck_id': truck_counter,
                'shiptos': truck_shiptos,
                'total_weight': current_weight,
                'rdc_id': rdc_id,
                'capacity_utilization': (current_weight / TRUCK_CAPACITY) * 100
            })
            truck_counter += 1

        # Remove processed shipments
        if indices_to_remove:
            remaining_shiptos = remaining_shiptos.drop(indices_to_remove).reset_index(drop=True)
        else:
            # Prevent infinite loop
            if len(remaining_shiptos) > 0:
                print(f"⚠️ Removing problematic shipment: {remaining_shiptos.iloc[0]['ShipToID']}")
                remaining_shiptos = remaining_shiptos.drop(0).reset_index(drop=True)

    return trucks

def nearest_neighbor_tsp(dist_matrix, start=0):
    """Nearest Neighbor TSP heuristic"""
    n = len(dist_matrix)
    unvisited = set(range(n))
    unvisited.remove(start)

    route = [start]
    current = start

    while unvisited:
        nearest = min(unvisited, key=lambda x: dist_matrix[current][x])
        route.append(nearest)
        unvisited.remove(nearest)
        current = nearest

    # Return to start
    route.append(start)
    return route

def farthest_insertion_tsp(dist_matrix, start=0):
    """Farthest Insertion TSP heuristic"""
    n = len(dist_matrix)

    # Start with the two farthest points from start
    unvisited = set(range(1, n))  # Exclude start point
    farthest = max(unvisited, key=lambda x: dist_matrix[start][x])

    route = [start, farthest, start]
    unvisited.remove(farthest)

    while unvisited:
        # Find the unvisited city farthest from the current route
        farthest_city = None
        max_min_distance = -1

        for city in unvisited:
            min_distance_to_route = min(dist_matrix[city][route_city] for route_city in route[:-1])
            if min_distance_to_route > max_min_distance:
                max_min_distance = min_distance_to_route
                farthest_city = city

        # Find the best position to insert the farthest city
        best_position = 1
        best_increase = float('inf')

        for i in range(1, len(route)):
            prev_city = route[i-1]
            next_city = route[i]

            increase = (dist_matrix[prev_city][farthest_city] +
                       dist_matrix[farthest_city][next_city] -
                       dist_matrix[prev_city][next_city])

            if increase < best_increase:
                best_increase = increase
                best_position = i

        # Insert the city at the best position
        route.insert(best_position, farthest_city)
        unvisited.remove(farthest_city)

    return route

def calculate_route_distance(route, dist_matrix):
    """Calculate total distance for a given route"""
    total_distance = 0
    for i in range(len(route) - 2):
        total_distance += dist_matrix[route[i]][route[i + 1]]
    return total_distance

def enhanced_tsp_solver(locations, dist_matrix):
    """Enhanced TSP solver with multiple heuristics"""
    n_locations = len(locations)

    if n_locations <= 1:
        return 0.0, [0] if n_locations == 1 else []

    if n_locations == 2:
        # Simple round trip for 2 locations
        return dist_matrix[0][1] * 2, [0, 1, 0]

    # Try multiple heuristics and pick the best
    best_distance = float('inf')
    best_route = None

    # Heuristic 1: Nearest Neighbor starting from depot (RDC)
    route_nn = nearest_neighbor_tsp(dist_matrix, start=0)
    distance_nn = calculate_route_distance(route_nn, dist_matrix)

    if distance_nn < best_distance:
        best_distance = distance_nn
        best_route = route_nn

    # Heuristic 2: Farthest Insertion (for larger problems)
    if n_locations > 5:
        try:
            route_fi = farthest_insertion_tsp(dist_matrix, start=0)
            distance_fi = calculate_route_distance(route_fi, dist_matrix)

            if distance_fi < best_distance:
                best_distance = distance_fi
                best_route = route_fi
        except:
            pass  # Fall back to nearest neighbor if farthest insertion fails

    return best_distance, best_route

def haversine_distance_vectorized(lat1, lon1, lat2, lon2):
    """Optimized Haversine distance calculation"""
    R = 6371  # Earth radius in km
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c

def get_state_enhanced(lat, lon):
    """Enhanced state determination with geopy and improved fallback"""
    try:
        from geopy.geocoders import Nominatim
        geolocator = Nominatim(user_agent="supply_chain_opt_final", timeout=5)
        location = geolocator.reverse(f"{lat}, {lon}", exactly_one=True)
        if location and location.raw.get('address'):
            address = location.raw['address']
            for key in ['state', 'administrative_area_level_1', 'province']:
                if key in address and address[key]:
                    state = address[key]
                    print(f"✅ Geopy: {state} for ({lat:.3f}, {lon:.3f})")
                    return state
    except:
        pass

    # Enhanced Australian state boundaries fallback
    if lat > -11:
        return "Northern Territory"
    elif lat > -25 and lon > 148:
        return "Queensland"
    elif lat > -29 and lon > 147:
        return "New South Wales"
    elif lat > -34 and lon > 143 and lon < 150:
        return "Victoria"
    elif lon < 130:
        return "Western Australia"
    elif lon < 143:
        return "South Australia"
    else:
        return "Tasmania"

def classify_location_type(lat, lon, state):
    """Enhanced urban/rural classification"""
    major_metros = {
        'New South Wales': [(-33.8688, 151.2093, 60), (-32.9267, 151.7767, 25)],
        'Victoria': [(-37.8136, 144.9631, 50), (-38.1478, 144.3606, 20)],
        'Queensland': [(-27.4698, 153.0251, 50), (-16.9186, 145.7781, 20)],
        'Western Australia': [(-31.9505, 115.8605, 40)],
        'South Australia': [(-34.9285, 138.6007, 30)],
        'Tasmania': [(-42.8821, 147.3272, 20)],
        'Northern Territory': [(-12.4634, 130.8456, 25)]
    }

    if state in major_metros:
        for city_lat, city_lon, radius in major_metros[state]:
            distance = haversine_distance_vectorized(lat, lon, city_lat, city_lon)
            if distance < radius:
                return "Urban"
    return "Rural"

---
# DATA PREPROCESSING WITH STANDARDIZATION

In [ ]:
print(f"\n📊 STEP 0: Data Preprocessing and Standardization...")

# Prepare features for K-means clustering
X_features = df_shipto_master[['Longitude', 'Latitude']].values
print(f"📍 Original coordinate ranges:")
print(f"   Longitude: [{X_features[:, 0].min():.3f}, {X_features[:, 0].max():.3f}]")
print(f"   Latitude: [{X_features[:, 1].min():.3f}, {X_features[:, 1].max():.3f}]")

# Apply StandardScaler for proper K-means clustering
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X_features)
print(f"✅ Data standardized using StandardScaler")


📊 STEP 0: Data Preprocessing and Standardization...
📍 Original coordinate ranges:
   Longitude: [114.627, 153.579]
   Latitude: [-42.973, -12.402]
✅ Data standardized using StandardScaler


---
# SCENARIO CONFIGURATIONS

In [ ]:
K_scenarios = [2, 3, 4, 5, 6, 7, 8]
print(f"🎯 Evaluating K scenarios: {K_scenarios}")

all_scenario_results = {}

# ========================================================================================
# MAIN SCENARIO EVALUATION LOOP
# ========================================================================================

for K in K_scenarios:
    print(f"\n" + "="*60)
    print(f"🎯 EVALUATING SCENARIO K = {K} RDCs")
    print("="*60)

    # ========================================================================================
    # STEP 1: STANDARDIZED K-MEANS CLUSTERING
    # ========================================================================================

    print(f"📍 Step 1: K-Means clustering with standardized data for K={K}...")

    # Apply K-Means on normalized data
    kmeans = KMeans(n_clusters=K, random_state=42, n_init=10, max_iter=300)
    cluster_labels = kmeans.fit_predict(X_normalized)

    # Transform centers back to original coordinate space
    rdc_centers_normalized = kmeans.cluster_centers_
    rdc_centers = scaler.inverse_transform(rdc_centers_normalized)

    # Add cluster assignment
    df_scenario = df_shipto_master.copy()
    df_scenario['RDC_Assignment'] = cluster_labels

    # Determine state and location type for each RDC
    rdc_states = []
    rdc_location_types = []

    print(f"✅ K-Means completed. Determining RDC characteristics...")
    for i, center in enumerate(rdc_centers):
        rdc_lon, rdc_lat = center
        rdc_state = get_state_enhanced(rdc_lat, rdc_lon)
        rdc_location_type = classify_location_type(rdc_lat, rdc_lon, rdc_state)

        rdc_states.append(rdc_state)
        rdc_location_types.append(rdc_location_type)

        cluster_data = df_scenario[df_scenario['RDC_Assignment'] == i]
        total_demand = cluster_data['Annual_KG_Demand_ShipTo'].sum()
        print(f"   RDC {i}: ({rdc_lon:.3f}, {rdc_lat:.3f}) - {rdc_state} ({rdc_location_type})")
        print(f"           Serves {len(cluster_data)} ShipToIDs, {total_demand:,.0f} KG")

    # ========================================================================================
    # STEP 2: BRANCH DELIVERY COSTS WITH ENHANCED CONSOLIDATION AND TSP
    # ========================================================================================

    print(f"\n🚛 Step 2: Branch Delivery Cost Calculation...")

    total_branch_delivery_cost = 0
    all_trucks_details = []

    for rdc_id in range(K):
        print(f"\n--- RDC {rdc_id} Analysis ({rdc_states[rdc_id]}) ---")

        rdc_shiptos = df_scenario[df_scenario['RDC_Assignment'] == rdc_id].copy()
        rdc_lon, rdc_lat = rdc_centers[rdc_id]
        rdc_state = rdc_states[rdc_id]

        if len(rdc_shiptos) == 0:
            continue

        print(f"🏢 RDC {rdc_id}: ({rdc_lon:.3f}, {rdc_lat:.3f}) in {rdc_state}")
        print(f"📦 Serves {len(rdc_shiptos)} ShipToIDs")
        print(f"⚖️ Total demand: {rdc_shiptos['Annual_KG_Demand_ShipTo'].sum():,.0f} KG")

        # ENHANCED: Use improved truck consolidation
        trucks = enhanced_truck_consolidation(rdc_shiptos, rdc_id, TRUCK_CAPACITY)

        print(f"🚚 Created {len(trucks)} truck trips with enhanced consolidation")

        # Enhanced TSP routing for each truck
        rdc_total_cost = 0
        for truck in trucks:
            # Build locations list
            locations = [(rdc_lon, rdc_lat)]  # RDC at index 0
            for shipto in truck['shiptos']:
                locations.append((shipto['Longitude'], shipto['Latitude']))

            n_locations = len(locations)

            # Pre-compute distance matrix
            dist_matrix = np.zeros((n_locations, n_locations))
            for i in range(n_locations):
                for j in range(i + 1, n_locations):
                    distance = haversine_distance_vectorized(
                        locations[i][1], locations[i][0],  # lat, lon
                        locations[j][1], locations[j][0]
                    )
                    dist_matrix[i][j] = distance
                    dist_matrix[j][i] = distance

            # ENHANCED: Use improved TSP solver
            total_distance, route = enhanced_tsp_solver(locations, dist_matrix)

            # Enhanced cost lookup
            intercept, slope, mode = lookup_cost_enhanced(rdc_state, total_distance, costcurve)
            print(f'Intercept: {intercept}')
            print(f'Slope: {slope}')
            print(f'Mode: {mode}')
            cost = intercept + slope * total_distance

            rdc_total_cost += cost
            all_trucks_details.append({
                'scenario_K': K,
                'rdc_id': rdc_id,
                'rdc_state': rdc_state,
                'truck_id': truck['truck_id'],
                'total_weight': truck['total_weight'],
                'total_distance': total_distance,
                'mode': mode,
                'cost': cost,
                'num_stops': len(truck['shiptos']),
                'capacity_utilization': truck['capacity_utilization']
            })

            print(f"   Truck {truck['truck_id']+1}: {len(truck['shiptos'])} stops, {truck['total_weight']:,.0f} KG")
            print(f"      Route distance: {total_distance:.1f} km, Mode: {mode}, Cost: ${cost:,.0f}")
            print(f"      Capacity utilization: {truck['capacity_utilization']:.1f}%")

        total_branch_delivery_cost += rdc_total_cost
        print(f"💰 RDC {rdc_id} total delivery cost: ${rdc_total_cost:,.0f}")

    print(f"\n🚛 Total Branch Delivery Cost K={K}: ${total_branch_delivery_cost:,.0f}")

    # ========================================================================================
    # STEP 3: HANDLING COSTS WITH CONSISTENT CBM CONVERSION
    # ========================================================================================

    print(f"\n📦 Step 3: Handling Cost Calculation...")

    total_handling_cost = 0
    handling_details = []

    for rdc_id in range(K):
        rdc_shiptos = df_scenario[df_scenario['RDC_Assignment'] == rdc_id]
        if len(rdc_shiptos) == 0:
            continue

        # Use consistent KG to CBM conversion
        total_cbm_handled = rdc_shiptos['Annual_KG_Demand_ShipTo'].sum() / ACTUAL_KG_PER_CBM

        rdc_state = rdc_states[rdc_id]
        location_type = rdc_location_types[rdc_id]

        # Apply location-based cost adjustment
        if location_type == "Urban":
            handling_cost_per_cbm = handling_cost_cbm_curr * URBAN_ADJUSTMENT_FACTOR
        else:
            handling_cost_per_cbm = handling_cost_cbm_curr * RURAL_ADJUSTMENT_FACTOR

        handling_cost_rdc = total_cbm_handled * handling_cost_per_cbm
        total_handling_cost += handling_cost_rdc

        handling_details.append({
            'rdc_id': rdc_id,
            'rdc_state': rdc_state,
            'location_type': location_type,
            'total_cbm': total_cbm_handled,
            'cost_per_cbm': handling_cost_per_cbm,
            'total_cost': handling_cost_rdc
        })

        print(f"📦 RDC {rdc_id} ({rdc_state}, {location_type}): {total_cbm_handled:,.0f} CBM × ${handling_cost_per_cbm:.1f} = ${handling_cost_rdc:,.0f}")

    print(f"\n📦 Total Handling Cost K={K}: ${total_handling_cost:,.0f}")

    # ========================================================================================
    # STEP 4: STORAGE COSTS WITH SQUARE ROOT LAW
    # ========================================================================================

    print(f"\n🏬 Step 4: Storage Cost Calculation with SRL...")
    print(f"📚 SRL Justification: Due to limited data on demand variability (σ_demand),")
    print(f"    supplier lead times, and ordering costs for individual RDCs, we apply the")
    print(f"    Square Root Law as a recognized method for estimating inventory changes.")
    print(f"    Current DC inventory ({avg_inv_cbm_curr:,} CBM) includes both cycle and safety stock.")

    # Apply Square Root Law
    total_inventory_cbm_new = avg_inv_cbm_curr * math.sqrt(K / 1)
    inventory_change_pct = ((total_inventory_cbm_new - avg_inv_cbm_curr) / avg_inv_cbm_curr) * 100

    print(f"📊 SRL Calculation: {avg_inv_cbm_curr:,} × √({K}/1) = {total_inventory_cbm_new:,.0f} CBM")
    print(f"📈 Inventory change: {inventory_change_pct:+.1f}% vs baseline")

    total_storage_cost = 0
    storage_details = []

    # Use consistent CBM for demand-based allocation
    total_system_demand_cbm = df_scenario['Annual_KG_Demand_ShipTo'].sum() / ACTUAL_KG_PER_CBM

    for rdc_id in range(K):
        rdc_shiptos = df_scenario[df_scenario['RDC_Assignment'] == rdc_id]
        if len(rdc_shiptos) == 0:
            continue

        # Allocate inventory based on CBM demand share
        rdc_demand_cbm = rdc_shiptos['Annual_KG_Demand_ShipTo'].sum() / ACTUAL_KG_PER_CBM
        demand_share = rdc_demand_cbm / total_system_demand_cbm
        inventory_at_rdc = total_inventory_cbm_new * demand_share

        # Apply location-based storage cost
        rdc_state = rdc_states[rdc_id]
        location_type = rdc_location_types[rdc_id]

        if location_type == "Urban":
            storage_cost_per_cbm = storage_cost_cbm_curr * URBAN_ADJUSTMENT_FACTOR
        else:
            storage_cost_per_cbm = storage_cost_cbm_curr * RURAL_ADJUSTMENT_FACTOR

        storage_cost_rdc = inventory_at_rdc * storage_cost_per_cbm
        total_storage_cost += storage_cost_rdc

        storage_details.append({
            'rdc_id': rdc_id,
            'rdc_state': rdc_state,
            'location_type': location_type,
            'demand_share': demand_share,
            'inventory_cbm': inventory_at_rdc,
            'cost_per_cbm': storage_cost_per_cbm,
            'total_cost': storage_cost_rdc
        })

        print(f"🏬 RDC {rdc_id} ({rdc_state}, {location_type}): {inventory_at_rdc:,.0f} CBM × ${storage_cost_per_cbm:.1f} = ${storage_cost_rdc:,.0f}")
        print(f"     Demand share: {demand_share*100:.1f}%")

    print(f"\n🏬 Total Storage Cost K={K}: ${total_storage_cost:,.0f}")

    # ========================================================================================
    # STEP 5: CALCULATE TRANSFER COSTS FROM ORIGINAL DC TO RDCs (HUB-AND-SPOKE MODEL)
    # ========================================================================================

    print(f"\n🔄 Step 5: Calculating Transfer Costs from Original DC to RDCs...")
    print(f"📍 Original DC: ({lon_ref}, {lat_ref}) in New South Wales")
    print(f"🚛 Hub-and-Spoke model: Original DC supplies all RDCs")

    total_transfer_cost = 0
    transfer_details = []

    # Get state of original DC for cost curve lookup
    dc_state = "New South Wales"  # Original DC is in NSW

    for rdc_id in range(K):
        rdc_shiptos = df_scenario[df_scenario['RDC_Assignment'] == rdc_id]
        if len(rdc_shiptos) == 0:
            continue

        rdc_lon, rdc_lat = rdc_centers[rdc_id]
        rdc_state = rdc_states[rdc_id]

        # Calculate Haversine distance from Original DC to RDC
        def haversine_distance_transfer(lat1, lon1, lat2, lon2):
            """Calculate Haversine distance between DC and RDC"""
            R = 6371  # Earth radius in km
            lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
            dlat = lat2 - lat1
            dlon = lon2 - lon1
            a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
            c = 2 * atan2(sqrt(a), sqrt(1 - a))
            return R * c

        transfer_distance = haversine_distance_transfer(lat_ref, lon_ref, rdc_lat, rdc_lon)

        # Calculate annual volume to transfer to this RDC
        annual_kg_to_rdc = rdc_shiptos['Annual_KG_Demand_ShipTo'].sum()

        # Calculate number of truck trips needed
        num_transfer_trips = math.ceil(annual_kg_to_rdc / TRUCK_CAPACITY)

        # Lookup cost curve for transfer from DC (NSW) based on distance
        try:
            intercept, slope, mode = lookup_cost_enhanced(dc_state, transfer_distance, costcurve)
            print(f'Interceptor: {intercept}')
            print(f'Slope: {slope}')
            print(f'Mode: {mode}')
            cost_per_trip = intercept + slope * transfer_distance
        except:
            # Fallback if lookup fails
            if transfer_distance <= 55:
                cost_per_trip = 225.2 + 0.9 * transfer_distance  # FTL NSW
                mode = 'FTL'
            else:
                cost_per_trip = 360.4 + 1.1 * transfer_distance  # LTL NSW
                mode = 'LTL'

        # Calculate total transfer cost for this RDC
        transfer_cost_rdc = num_transfer_trips * cost_per_trip
        total_transfer_cost += transfer_cost_rdc

        # Store transfer details
        transfer_details.append({
            'rdc_id': rdc_id,
            'rdc_state': rdc_state,
            'rdc_coordinates': (rdc_lon, rdc_lat),
            'transfer_distance': transfer_distance,
            'annual_kg': annual_kg_to_rdc,
            'num_trips': num_transfer_trips,
            'cost_per_trip': cost_per_trip,
            'mode': mode,
            'total_cost': transfer_cost_rdc
        })

        print(f"🚛 Transfer to RDC {rdc_id} ({rdc_state}):")
        print(f"    Distance: {transfer_distance:.1f} km")
        print(f"    Annual volume: {annual_kg_to_rdc:,.0f} KG")
        print(f"    Number of trips: {num_transfer_trips}")
        print(f"    Cost per trip: ${cost_per_trip:.2f} ({mode})")
        print(f"    Total transfer cost: ${transfer_cost_rdc:,.0f}")

    print(f"\n🔄 Total Transfer Cost K={K}: ${total_transfer_cost:,.0f}")
    print(f"📊 Transfer Cost Justification:")
    print(f"    • Hub-and-Spoke model: Original DC supplies all RDCs")
    print(f"    • {TRUCK_CAPACITY:,} KG capacity trucks used for transfers")
    print(f"    • Cost curves applied based on DC state (NSW) and transfer distances")
    print(f"    • {sum(detail['num_trips'] for detail in transfer_details)} total transfer trips annually")

    # ========================================================================================
    # STEP 6: CALCULATE TOTAL SCENARIO COST
    # ========================================================================================

    total_cost_scenario = total_branch_delivery_cost + total_handling_cost + total_storage_cost + total_transfer_cost
    savings_vs_baseline = baseline_total_cost - total_cost_scenario
    savings_percentage = (savings_vs_baseline / baseline_total_cost * 100) if baseline_total_cost > 0 else 0

    print(f"\n💰 SCENARIO K={K} TOTAL COST BREAKDOWN:")
    print(f"   🚛 Branch Delivery: ${total_branch_delivery_cost:,.0f} ({total_branch_delivery_cost/total_cost_scenario*100:.1f}%)")
    print(f"   📦 Handling: ${total_handling_cost:,.0f} ({total_handling_cost/total_cost_scenario*100:.1f}%)")
    print(f"   🏬 Storage: ${total_storage_cost:,.0f} ({total_storage_cost/total_cost_scenario*100:.1f}%)")
    print(f"   🔄 Transfer: ${total_transfer_cost:,.0f} ({total_transfer_cost/total_cost_scenario*100:.1f}%)")
    print(f"   🎯 TOTAL: ${total_cost_scenario:,.0f}")
    print(f"   💾 Savings vs Baseline: ${savings_vs_baseline:,.0f} ({savings_percentage:+.1f}%)")

    # Store comprehensive results
    all_scenario_results[K] = {
        'K': K,
        'rdc_centers': rdc_centers,
        'rdc_states': rdc_states,
        'rdc_location_types': rdc_location_types,
        'total_cost': total_cost_scenario,
        'branch_delivery_cost': total_branch_delivery_cost,
        'handling_cost': total_handling_cost,
        'storage_cost': total_storage_cost,
        'transfer_cost': total_transfer_cost,
        'savings_vs_baseline': savings_vs_baseline,
        'savings_percentage': savings_percentage,
        'trucks_details': all_trucks_details,
        'handling_details': handling_details,
        'storage_details': storage_details,
        'transfer_details': transfer_details,
        'scenario_data': df_scenario,
        'total_inventory_cbm': total_inventory_cbm_new
    }

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
      Capacity utilization: 99.7%
Intercept: 360.4
Slope: 1.1
Mode: LTL
   Truck 77: 10 stops, 19,989 KG
      Route distance: 4056.5 km, Mode: LTL, Cost: $4,823
      Capacity utilization: 99.9%
Intercept: 360.4
Slope: 1.1
Mode: LTL
   Truck 78: 10 stops, 19,837 KG
      Route distance: 3171.1 km, Mode: LTL, Cost: $3,849
      Capacity utilization: 99.2%
Intercept: 360.4
Slope: 1.1
Mode: LTL
   Truck 79: 11 stops, 19,996 KG
      Route distance: 5192.7 km, Mode: LTL, Cost: $6,072
      Capacity utilization: 100.0%
Intercept: 360.4
Slope: 1.1
Mode: LTL
   Truck 80: 11 stops, 19,751 KG
      Route distance: 3253.7 km, Mode: LTL, Cost: $3,939
      Capacity utilization: 98.8%
Intercept: 360.4
Slope: 1.1
Mode: LTL
   Truck 81: 12 stops, 19,979 KG
      Route distance: 4255.6 km, Mode: LTL, Cost: $5,042
      Capacity utilization: 99.9%
Intercept: 360.4
Slope: 1.1
Mode: LTL
   Truck 82: 12 stops, 19,297 KG
      Route distance: 3360.

---
# COMPREHENSIVE VISUALIZATION WITH PLOTLY

In [ ]:
print(f"\n" + "="*80)
print("📊 CREATING COMPREHENSIVE PLOTLY VISUALIZATIONS")
print("="*80)

# Prepare scenario comparison data
scenario_data = []
for K, results in all_scenario_results.items():
    scenario_data.append({
        'K': K,
        'Total_Cost': results['total_cost'],
        'Branch_Delivery': results['branch_delivery_cost'],
        'Handling': results['handling_cost'],
        'Storage': results['storage_cost'],
        'Transfer': results['transfer_cost'],
        'Savings_vs_Baseline': results['savings_vs_baseline'],
        'Savings_Percentage': results['savings_percentage']
    })

scenario_df = pd.DataFrame(scenario_data)
scenario_df


📊 CREATING COMPREHENSIVE PLOTLY VISUALIZATIONS


,K,Total_Cost,Branch_Delivery,Handling,Storage,Transfer,Savings_vs_Baseline,Savings_Percentage
0,2,9.621547e+05,273904.967245,104640.443654,4.424480e+05,141161.228423,2.116272e+06,68.745250
1,3,9.701880e+05,156336.506350,104640.443654,5.418860e+05,167325.139265,2.108239e+06,68.484293
2,4,9.949627e+05,98698.686109,104640.443654,6.257160e+05,165907.608597,2.083464e+06,67.679508
3,5,1.227106e+06,63402.175518,128485.644164,8.589884e+05,176230.215983,1.851321e+06,60.138524
4,6,1.496213e+06,61350.909272,150885.714232,1.105023e+06,178953.194830,1.582214e+06,51.396829
5,7,1.645019e+06,59056.923702,157559.469165,1.246353e+06,182049.292995,1.433408e+06,46.562997
6,8,1.733815e+06,57281.508299,157559.469165,1.332408e+06,186566.419423,1.344612e+06,43.678532


In [ ]:
# nhân 5 lên
scenario_df['Branch_Delivery'] *= 5
scenario_df['Transfer'] *= 5
scenario_df['Handling'] *= 5
scenario_df['Total_Cost'] = scenario_df['Branch_Delivery'] + scenario_df['Handling'] + scenario_df['Storage'] + scenario_df['Transfer']
scenario_df['Savings_vs_Baseline'] = baseline_total_cost - scenario_df['Total_Cost']
scenario_df['Savings_Percentage'] = scenario_df['Savings_vs_Baseline'] / baseline_total_cost * 100

---
## 1. Enhanced Stacked Bar Chart with Transfer costs

In [ ]:
fig_cost_breakdown = go.Figure()

fig_cost_breakdown.add_trace(go.Bar(
    name='Branch Delivery',
    x=scenario_df['K'],
    y=scenario_df['Branch_Delivery'],
    marker_color='#FF6B6B',
    text=[f'${val:,.0f}' for val in scenario_df['Branch_Delivery']],
    textposition='inside',
    textfont_color='white'
))

fig_cost_breakdown.add_trace(go.Bar(
    name='Handling',
    x=scenario_df['K'],
    y=scenario_df['Handling'],
    marker_color='#4ECDC4',
    text=[f'${val:,.0f}' for val in scenario_df['Handling']],
    textposition='inside',
    textfont_color='white'
))

fig_cost_breakdown.add_trace(go.Bar(
    name='Storage',
    x=scenario_df['K'],
    y=scenario_df['Storage'],
    marker_color='#45B7D1',
    text=[f'${val:,.0f}' for val in scenario_df['Storage']],
    textposition='inside',
    textfont_color='white'
))

fig_cost_breakdown.add_trace(go.Bar(
    name='Transfer',
    x=scenario_df['K'],
    y=scenario_df['Transfer'],
    marker_color='#96CEB4',
    text=[f'${val:,.0f}' for val in scenario_df['Transfer']],
    textposition='inside',
    textfont_color='white'
))

# Add baseline reference line
fig_cost_breakdown.add_hline(
    y=baseline_total_cost,
    line_dash="dash",
    line_color="red",
    line_width=3,
    annotation_text=f"Baseline: ${baseline_total_cost:,.0f}",
    annotation_position="top right"
)

fig_cost_breakdown.update_layout(
    title='<b>Annual Cost Breakdown by RDC Scenarios (Including Transfer Costs)</b>',
    xaxis_title='Number of RDCs (K)',
    yaxis_title='Annual Cost (AUD)',
    barmode='stack',
    height=600,
    font=dict(size=12)
)

fig_cost_breakdown.show()

---
## 2. Savings Analysis

In [ ]:
fig_savings = go.Figure()

colors = ['green' if x > 0 else 'red' for x in scenario_df['Savings_vs_Baseline']]

fig_savings.add_trace(go.Bar(
    x=scenario_df['K'],
    y=scenario_df['Savings_vs_Baseline'],
    marker_color=colors,
    text=[f'${val:,.0f}<br>({pct:+.1f}%)' for val, pct in
          zip(scenario_df['Savings_vs_Baseline'], scenario_df['Savings_Percentage'])],
    textposition='outside',
    name='Cost Savings vs Baseline'
))

fig_savings.add_hline(y=0, line_dash="solid", line_color="black", line_width=1)

fig_savings.update_layout(
    title='<b>Cost Savings vs Baseline by RDC Scenarios</b>',
    xaxis_title='Number of RDCs (K)',
    yaxis_title='Savings (AUD)',
    height=500
)

fig_savings.show()

---
## 3. Transfer Cost Analysis

In [ ]:
fig_transfer_analysis = go.Figure()

transfer_costs = [results['transfer_cost'] for results in all_scenario_results.values()]
transfer_trips = []
for results in all_scenario_results.values():
    total_trips = sum(detail['num_trips'] for detail in results.get('transfer_details', []))
    transfer_trips.append(total_trips)

fig_transfer_analysis.add_trace(go.Scatter(
    x=scenario_df['K'],
    y=transfer_costs,
    mode='lines+markers',
    name='Transfer Cost',
    line=dict(width=3, color='green'),
    marker=dict(size=10),
    yaxis='y'
))

fig_transfer_analysis.add_trace(go.Scatter(
    x=scenario_df['K'],
    y=transfer_trips,
    mode='lines+markers',
    name='Transfer Trips',
    line=dict(width=2, color='orange'),
    marker=dict(size=8),
    yaxis='y2'
))

fig_transfer_analysis.update_layout(
    title='<b>Transfer Cost and Trip Analysis by Number of RDCs</b>',
    xaxis_title='Number of RDCs (K)',
    yaxis=dict(title='Transfer Cost (AUD)', side='left'),
    yaxis2=dict(title='Number of Transfer Trips', side='right', overlaying='y'),
    height=500
)

fig_transfer_analysis.show()

---
## 4. Network Maps for Each Scenario

In [ ]:
for K, results in all_scenario_results.items():
    print(f"🗺️ Creating network map for K={K} scenario...")

    df_scenario = results['scenario_data']
    rdc_centers = results['rdc_centers']
    rdc_states = results['rdc_states']
    rdc_location_types = results['rdc_location_types']

    fig_network = go.Figure()

    # Plot original DC
    fig_network.add_trace(go.Scatter(
        x=[lon_ref],
        y=[lat_ref],
        mode='markers+text',
        marker=dict(symbol='diamond', size=25, color='red',
                   line=dict(width=3, color='white')),
        text=['Original DC<br>NSW'],
        textposition="top center",
        name='Original DC',
        hovertemplate='Original DC (NSW)<extra></extra>'
    ))

    # Plot RDCs
    fig_network.add_trace(go.Scatter(
        x=rdc_centers[:, 0],
        y=rdc_centers[:, 1],
        mode='markers+text',
        marker=dict(symbol='star', size=20, color='blue',
                   line=dict(width=2, color='white')),
        text=[f'RDC {i}<br>{rdc_states[i]}<br>({rdc_location_types[i]})' for i in range(K)],
        textposition="bottom center",
        name='RDCs',
        hovertemplate='%{text}<extra></extra>'
    ))

    # Plot transfer connections
    for i, center in enumerate(rdc_centers):
        fig_network.add_trace(go.Scatter(
            x=[lon_ref, center[0]],
            y=[lat_ref, center[1]],
            mode='lines',
            line=dict(color='red', width=2, dash='dash'),
            name=f'Transfer to RDC {i}',
            showlegend=False,
            hovertemplate='Transfer route<extra></extra>'
        ))

    # Plot ShipToIDs by cluster
    colors = px.colors.qualitative.Set1
    for rdc_id in range(K):
        cluster_data = df_scenario[df_scenario['RDC_Assignment'] == rdc_id]

        if len(cluster_data) > 0:
            fig_network.add_trace(go.Scatter(
                x=cluster_data['Longitude'],
                y=cluster_data['Latitude'],
                mode='markers',
                marker=dict(
                    color=colors[rdc_id % len(colors)],
                    size=np.clip(cluster_data['Annual_KG_Demand_ShipTo']/2000, 4, 15),
                    opacity=0.7,
                    line=dict(width=1, color='white')
                ),
                text=[f"<b>{row['ShipToID']}</b><br>RDC: {row['RDC_Assignment']} ({rdc_states[row['RDC_Assignment']]})<br>KG: {row['Annual_KG_Demand_ShipTo']:,.0f}"
                      for _, row in cluster_data.iterrows()],
                hovertemplate='%{text}<extra></extra>',
                name=f'RDC {rdc_id} Zone ({len(cluster_data)} locations)',
                showlegend=True
            ))

    savings = results['savings_vs_baseline']
    fig_network.update_layout(
        title=f'<b>Hub-and-Spoke Network: K={K} RDCs</b><br>' +
              f'Total Cost: ${results["total_cost"]:,.0f} | ' +
              f'Transfer Cost: ${results["transfer_cost"]:,.0f} | ' +
              f'Savings: ${savings:,.0f} ({results["savings_percentage"]:+.1f}%)',
        xaxis_title='Longitude (°E)',
        yaxis_title='Latitude (°S)',
        height=700,
        showlegend=True
    )

    fig_network.show()

🗺️ Creating network map for K=2 scenario...


🗺️ Creating network map for K=3 scenario...


🗺️ Creating network map for K=4 scenario...


🗺️ Creating network map for K=5 scenario...


🗺️ Creating network map for K=6 scenario...


🗺️ Creating network map for K=7 scenario...


🗺️ Creating network map for K=8 scenario...
